In [ ]:
!pip install python-pptx

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import os
from datetime import datetime
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# PowerPoint imports
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.enum.shapes import MSO_SHAPE

print("All libraries imported successfully!")

In [ ]:
from google.colab import files
import os
from pptx import Presentation

# Create Raw folder if it doesn't exist
if not os.path.exists('Raw'):
    os.makedirs('Raw')

print("Please upload your Bonds*.xlsx, History*.xlsx, AND your PowerPoint template (.pptx) file.")
uploaded = files.upload()

pptx_filename = None
for fname in uploaded:
    if fname.lower().endswith('.pptx'):
        pptx_filename = fname
        prs = Presentation(pptx_filename)
        print(f"Loaded PowerPoint template: {pptx_filename} with {len(prs.slides)} slides.")
    else:
        os.rename(fname, f'Raw/{fname}')
        print(f"Moved {fname} to Raw/")

if pptx_filename is None:
    print("WARNING: No PowerPoint template (.pptx) uploaded!")
else:
    print(f"PowerPoint template ready: {pptx_filename}")

print("\nFiles in Raw folder:", os.listdir('Raw'))

In [ ]:
def load_bonds_history(raw_folder='Raw'):
    """Load bonds and history data from Excel files."""
    import pandas as pd
    import os

    bond_files = [f for f in os.listdir(raw_folder) if f.startswith('Bonds') and f.endswith('.xlsx')]
    hist_files = [f for f in os.listdir(raw_folder) if f.startswith('History') and f.endswith('.xlsx')]

    if not bond_files:
        print("No Bonds files found in Raw/")
        bonds = pd.DataFrame()
    else:
        bonds = pd.concat([pd.read_excel(os.path.join(raw_folder, f)) for f in bond_files], ignore_index=True)

    if not hist_files:
        print("No History files found in Raw/")
        history = pd.DataFrame()
    else:
        history = pd.concat([pd.read_excel(os.path.join(raw_folder, f)) for f in hist_files], ignore_index=True)

    return bonds, history

# Load data
bonds, history = load_bonds_history()
print(f"Loaded {len(bonds)} bonds and {len(history)} history records")
print("\nBonds columns:", list(bonds.columns) if not bonds.empty else "No Bonds data")
print("\nHistory columns:", list(history.columns) if not history.empty else "No History data")


In [ ]:
# Assign maturity buckets to match Treasury field exactly
bonds['MaturityBucket'] = bonds['Treasury'].astype(str).str.lower()

import re

def build_coupon_date_to_ticker(history):
    mapping = {}
    pattern = r"^([A-Z]+) ([\d.]+% \d+/\d{4})"
    for col in history.columns:
        m = re.match(pattern, col)
        if m:
            ticker, coup_date = m.group(1), m.group(2)
            mapping[coup_date] = ticker
            coupon, date = coup_date.split('%')[0], coup_date.split('%')[1].strip()
            try:
                mapping[f"{float(coupon):.1f}% {date}"] = ticker
                mapping[f"{float(coupon):.3f}% {date}"] = ticker
            except Exception:
                pass
    return mapping

coupon_date_to_ticker = build_coupon_date_to_ticker(history)

def make_bond_label(row):
    if 'Description' in row and isinstance(row['Description'], str):
        desc = row['Description']
        desc_parts = desc.split()
        if len(desc_parts) >= 1:
            ticker_candidate = desc_parts[0].upper()
            if ticker_candidate != "UNKNOWN":
                coupon = f"{row['Coupon']:.3f}" if pd.notna(row['Coupon']) else '0.000'
                maturity_date = row['Maturity Date']
                if pd.notna(maturity_date):
                    maturity = pd.to_datetime(maturity_date) if isinstance(maturity_date, str) else maturity_date
                    short_date = f"{maturity.month}/{maturity.year}"
                else:
                    short_date = 'Unknown'
                return f"{ticker_candidate} {coupon}% {short_date}"
    coupon = f"{row['Coupon']:.3f}" if pd.notna(row['Coupon']) else '0.000'
    maturity_date = row['Maturity Date']
    if pd.notna(maturity_date):
        maturity = pd.to_datetime(maturity_date) if isinstance(maturity_date, str) else maturity_date
        short_date = f"{maturity.month}/{maturity.year}"
    else:
        short_date = 'Unknown'
    coup_date_1dp = f"{float(row['Coupon']):.1f}% {short_date}"
    coup_date_3dp = f"{float(row['Coupon']):.3f}% {short_date}"
    ticker = coupon_date_to_ticker.get(coup_date_1dp) or coupon_date_to_ticker.get(coup_date_3dp) or "Unknown"
    return f"{ticker} {coupon}% {short_date}"

bonds['BondLabel'] = bonds.apply(make_bond_label, axis=1)
bonds['Ticker'] = bonds['BondLabel'].str.split().str[0].str.upper()

print("Maturity buckets and bond labels created using Treasury as bucket:")
print(bonds[['BondLabel', 'Ticker', 'Treasury', 'MaturityBucket']].head(10))

In [ ]:
# On-the-Run toggle cell

on_the_run_input = input("Filter to on-the-run bonds only? (yes/no): ").strip().lower()

if on_the_run_input in ['yes', 'y']:
    print("Filtering to on-the-run bonds only...")
    def latest_only(group):
        group = group.copy()
        group['Issue Date'] = pd.to_datetime(group['Issue Date'])
        return group.sort_values('Issue Date', ascending=False).head(1)
    filtered_bonds = bonds.groupby(['Ticker', 'MaturityBucket'], group_keys=False).apply(latest_only)
    bonds = filtered_bonds.reset_index(drop=True)
    print(f"After filtering: {len(bonds)} bonds remain.")
else:
    print("No filtering applied. Using full bond list.")

print(bonds[['BondLabel', 'Ticker', 'Treasury', 'MaturityBucket', 'Issue Date']])


In [ ]:
# Dynamically extract available tickers from BondLabel/Ticker column
def extract_valid_tickers(bonds_df):
    return sorted(bonds_df['Ticker'].unique())

AVAILABLE_TICKERS = extract_valid_tickers(bonds)

def find_bond_ticker(user_input, bonds_df):
    ticker = user_input.strip().upper()
    tickers = extract_valid_tickers(bonds_df)
    matches = bonds_df[bonds_df['Ticker'] == ticker]
    if matches.empty:
        print("No matches found. Available tickers:")
        print(tickers)
        return None
    print(f"Selected ticker: {ticker}")
    return ticker, matches

user_input = input(f"Enter a bond ticker {AVAILABLE_TICKERS}: ")
result = find_bond_ticker(user_input, bonds)
if result is None:
    print("Try again with a different ticker.")
else:
    ticker, target_bonds = result
    print(f"Found {len(target_bonds)} bonds for {ticker}:")
    print(target_bonds[['BondLabel', 'MaturityBucket']].drop_duplicates())


In [ ]:
def index_targets_and_peer_groups(target_bonds, bonds):
    """Index target bonds and assign peer groups"""
    indexed_targets = []
    buckets = target_bonds['MaturityBucket'].dropna().unique()
    idx_counter = 1

    for bucket in sorted(buckets):
        bucket_targets = target_bonds[target_bonds['MaturityBucket'] == bucket]
        peer_group = bonds[bonds['MaturityBucket'] == bucket].copy()

        for _, target_row in bucket_targets.iterrows():
            bond_label = target_row['BondLabel']
            # Exclude the target from its own peer group
            peer_group_only = peer_group[peer_group['BondLabel'] != bond_label].copy()
            peer_group_only['peer_idx'] = idx_counter

            indexed_targets.append({
                'idx': idx_counter,
                'bucket': bucket,
                'BondLabel': bond_label,
                'Description': target_row['Description'],
                'peer_group': peer_group_only
            })
            idx_counter += 1

    return indexed_targets

# Create indexed targets
indexed_targets = index_targets_and_peer_groups(target_bonds, bonds)

print(f"Created {len(indexed_targets)} indexed target bonds:")
for t in indexed_targets:
    print(f"  idx: {t['idx']}, bucket: {t['bucket']}, bond: {t['BondLabel']}, peers: {len(t['peer_group'])}")


In [ ]:
def calc_peer_avg_series(indexed_targets, history):
    for entry in indexed_targets:
        peer_group = entry['peer_group']
        peer_descs = peer_group['Description'].tolist()
        tspread_cols = [desc + ' T-Spread' for desc in peer_descs]
        dates = pd.to_datetime(history['Date'])
        drop_idx = ~dates.duplicated(keep='first')
        dates_nodup = dates[drop_idx].reset_index(drop=True)
        if tspread_cols:
            existing_cols = [col for col in tspread_cols if col in history.columns]
            if existing_cols:
                peer_tspreads_nodup = history.loc[drop_idx, existing_cols].reset_index(drop=True)
                # Only calculate mean when at least two non-NaN peers are present
                peer_avg_series = peer_tspreads_nodup.apply(
                    lambda row: np.nanmean(row.values) if np.sum(~np.isnan(row.values)) >= 2 else np.nan, axis=1
                )
            else:
                peer_avg_series = pd.Series(index=dates_nodup, dtype=float)
        else:
            peer_avg_series = pd.Series(index=dates_nodup, dtype=float)
        peer_dates = dates_nodup[~peer_avg_series.isna().to_numpy()]
        entry['peer_avg_series'] = peer_avg_series
        entry['peer_dates'] = peer_dates
    return indexed_targets

# Apply function
calc_peer_avg_series(indexed_targets, history)


In [ ]:
def calc_true_peer_avg_start(indexed_targets, history):
    start_dates_dict = {}

    for entry in indexed_targets:
        bucket = entry['bucket']
        bond_label = entry['BondLabel']
        peer_dates = entry['peer_dates']
        desc = entry['Description']
        target_col = desc + ' T-Spread'

        print(f"\n[{bucket}] {bond_label}")

        if target_col not in history.columns:
            print(f"  SKIP (target_col not found)")
            continue

        target_hist = history[['Date', target_col]].dropna()
        if target_hist.empty:
            print(f"  SKIP (no target trade dates)")
            continue

        target_hist['Date'] = pd.to_datetime(target_hist['Date'])
        target_trade_dates = target_hist['Date']
        target_days = target_trade_dates.dt.date

        print(f"  peer_dates: {list(peer_dates)[:10]} (total {len(peer_dates)})")
        print(f"  target_trade_dates: {list(target_trade_dates)[:10]} (total {len(target_trade_dates)})")

        if len(peer_dates) == 0:
            print(f"  SKIP (no peer avg values)")
            continue

        peer1_date = peer_dates.min()
        peer1_day = peer1_date.date()

        # Start date logic
        if any(target_days == peer1_day):
            first_of_month = pd.Timestamp(year=peer1_date.year, month=peer1_date.month, day=1)
            first_day = first_of_month.date()
            if any(target_days == first_day):
                true_start = first_of_month
            else:
                after_first = target_trade_dates[target_trade_dates > first_of_month]
                if len(after_first) > 0:
                    true_start = after_first.min()
                else:
                    true_start = peer1_date
        else:
            after_peer = target_trade_dates[target_trade_dates > peer1_date]
            if len(after_peer) > 0:
                true_start = after_peer.min()
            else:
                true_start = target_trade_dates.min()

        entry['start_date'] = true_start
        start_dates_dict[(bucket, bond_label)] = true_start
        print(f"  Start = {true_start}")

    print("\nFinal start_dates_dict:", start_dates_dict)
    return start_dates_dict

# Calculate and store start dates
start_dates_dict = calc_true_peer_avg_start(indexed_targets, history)


In [ ]:
import os
from io import BytesIO

def save_all_graphs(indexed_targets, history, start_dates_dict, output_dir="charts"):
    """
    For each indexed target, saves:
      - peer group chart (plot_peer_group_fixed)
      - peer average chart (plot_peer_avg_fixed)
    as PNG images in output_dir.
    Returns list of (peer_group_png, peer_avg_png) for each entry.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    chart_file_pairs = []
    for entry in indexed_targets:
        label_safe = entry['BondLabel'].replace(" ", "_").replace("%", "pct").replace("/", "_")

        # Peer group chart
        peer_group_png = os.path.join(output_dir, f"{label_safe}_peer_group.png")
        with open(peer_group_png, "wb") as f1:
            buf1 = BytesIO()
            plot_peer_group_fixed(entry, history, start_dates_dict, buf1)
            f1.write(buf1.getbuffer())
            buf1.close()

        # Peer average chart
        peer_avg_png = os.path.join(output_dir, f"{label_safe}_peer_avg.png")
        with open(peer_avg_png, "wb") as f2:
            buf2 = BytesIO()
            plot_peer_avg_fixed(entry, history, start_dates_dict, buf2)
            f2.write(buf2.getbuffer())
            buf2.close()

        chart_file_pairs.append((peer_group_png, peer_avg_png))

    print(f"Saved {len(chart_file_pairs)} pairs of chart images to '{output_dir}'")
    return chart_file_pairs

# Usage after setup:
# chart_file_pairs = save_all_graphs(indexed_targets, history, start_dates_dict)


In [ ]:
def plot_peer_avg_fixed(entry, history, start_dates_dict, buffer):
    """Peer average chart with expanded vertical space."""
    import matplotlib.pyplot as plt
    import numpy as np

    fig, ax = plt.subplots(figsize=(10, 4), dpi=100)
    plt.style.use('default')
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')

    debug_key = (entry['bucket'], entry['BondLabel'])
    start_date = start_dates_dict.get(debug_key)
    if start_date is None:
        plt.close()
        return

    description = entry['Description']
    peer_group = entry['peer_group']
    peer_descs = peer_group['Description'].tolist()
    tspread_cols = [desc + ' T-Spread' for desc in peer_descs]

    target_col = description + ' T-Spread'
    if target_col not in history.columns:
        plt.close()
        return

    target_hist = history[['Date', target_col]].dropna()
    if target_hist.empty:
        plt.close()
        return

    target_hist['Date'] = pd.to_datetime(target_hist['Date'])
    t_dates = target_hist['Date']
    t_vals = target_hist[target_col].values

    dates = pd.to_datetime(history['Date'])
    drop_idx = ~dates.duplicated(keep='first')
    dates_nodup = dates[drop_idx].reset_index(drop=True)

    if tspread_cols:
        existing_cols = [col for col in tspread_cols if col in history.columns]
        if existing_cols:
            peer_tspreads_nodup = history[existing_cols].loc[drop_idx].reset_index(drop=True)
            peer_avg_series = peer_tspreads_nodup.mean(axis=1)
        else:
            peer_avg_series = pd.Series(index=dates_nodup, dtype=float)
    else:
        peer_avg_series = pd.Series(index=dates_nodup, dtype=float)

    peer_dates = dates_nodup[peer_avg_series.notna()]
    peer_vals = peer_avg_series[peer_avg_series.notna()].values

    if len(peer_dates) > 0:
        last_date = max(t_dates.max(), peer_dates.max())
    else:
        last_date = t_dates.max()

    forced_index = pd.date_range(start_date, last_date, freq='D')
    t_interp = np.interp(forced_index.astype(np.int64), t_dates.values.astype(np.int64), t_vals)

    if len(peer_dates) > 0:
        ax.plot(peer_dates, peer_vals, label='Peer Avg', color='#2171b5', linewidth=2.5, linestyle='--', alpha=0.9)

    ax.plot(forced_index, t_interp, label=f"{entry['BondLabel']} (Target)", color='#111111', linewidth=3.5)

    all_y = np.concatenate([t_interp, peer_vals]) if len(peer_vals) > 0 else t_interp
    ymin = 10 * np.floor(np.nanmin(all_y) / 10)
    ymax = 10 * np.ceil(np.nanmax(all_y) / 10)
    ax.set_ylim(ymin, ymax)
    ax.set_xlim(forced_index[0], forced_index[-1])

    y_ticks = np.linspace(ymin, ymax, 6)
    ax.set_yticks(y_ticks)
    ax.set_ylabel('T-Spread (bps)', fontsize=11)
    ax.set_xlabel('Date', fontsize=11)

    num_ticks = 5
    tick_idxs = np.linspace(0, len(forced_index) - 1, num_ticks, dtype=int)
    ax.set_xticks(forced_index[tick_idxs])
    ax.set_xticklabels([d.strftime('%-m/%-d/%y') for d in forced_index[tick_idxs]], fontsize=9, rotation=45, ha='right')

    ax.grid(True, alpha=0.2, linestyle='-', linewidth=0.5)
    ax.set_axisbelow(True)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.18), ncol=2, frameon=False, fontsize=10)

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)

    fig.text(0.98, 0.02, 'Source: Bondway.ai, CAVU Securities', fontsize=8, color='#666', ha='right')

    plt.tight_layout()
    plt.subplots_adjust(top=0.88, bottom=0.15, left=0.1, right=0.98)
    plt.savefig(buffer, format='png', dpi=100, bbox_inches='tight')
    plt.close()


In [ ]:
def plot_peer_group_fixed(entry, history, start_dates_dict, buffer):
    """Peer group chart with expanded vertical space."""
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 4), dpi=100)
    plt.style.use('default')
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')

    debug_key = (entry['bucket'], entry['BondLabel'])
    start_date = start_dates_dict.get(debug_key)
    if start_date is None:
        plt.close()
        return

    # Color palette for lines
    colors = ["#2894c7", "#84ba5b", "#ea9b31", "#bc7dbb", "#7b868c", "#d07676"]

    # Get peer bonds (limit to 5) plus target bond
    peer_descs = entry['peer_group']['Description'].tolist()[:5]
    plot_descs = peer_descs + [entry['Description']]

    for idx, desc in enumerate(plot_descs):
        col = desc + ' T-Spread'
        if col in history.columns:
            d = history[['Date', col]].dropna()
            d['Date'] = pd.to_datetime(d['Date'])
            if desc == entry['Description']:
                ax.plot(d['Date'], d[col], label=desc + " (Target)", color='#111111', linewidth=3.5)
            else:
                ax.plot(d['Date'], d[col], label=desc, color=colors[idx % len(colors)], linewidth=2)

    ax.set_ylabel('T-Spread (bps)', fontsize=11)
    ax.set_xlabel('Date', fontsize=11)
    ax.grid(True, alpha=0.2, linestyle='-', linewidth=0.5)
    ax.set_axisbelow(True)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.18), ncol=2, frameon=False, fontsize=10)
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)
    fig.text(0.98, 0.02, 'Source: Bondway.ai, CAVU Securities', fontsize=8, color='#666', ha='right')
    plt.tight_layout()
    plt.subplots_adjust(top=0.88, bottom=0.15, left=0.1, right=0.98)
    plt.savefig(buffer, format='png', dpi=100, bbox_inches='tight')
    plt.close()


In [ ]:
# Generate all chart images as PNGs and list them for verification

# Generate and save
chart_file_pairs = save_all_graphs(indexed_targets, history, start_dates_dict)

# List all generated PNGs
import os
def list_generated_graphs(output_dir="charts"):
    pngs = sorted([f for f in os.listdir(output_dir) if f.endswith(".png")])
    for f in pngs:
        print(f)
    return pngs

list_generated_graphs("charts")


In [ ]:
import os
bad_files = [
    "charts/MCD_3.500pct_7_2027_peer_group.png",
    "charts/MCD_3.500pct_7_2027_peer_avg.png",
    "charts/MCD_4.600pct_5_2030_peer_group.png",
    "charts/MCD_4.600pct_5_2030_peer_avg.png"
]
for f in bad_files:
    if os.path.exists(f):
        os.remove(f)


In [ ]:
from pptx import Presentation

def get_placeholder_positions(template_pptx_path, slide_idx):
    prs = Presentation(template_pptx_path)
    slide = prs.slides[slide_idx]
    positions = []
    for shape in slide.shapes:
        if not shape.has_text_frame:
            positions.append((shape.left, shape.top, shape.width, shape.height))
    return positions

def add_graphs_automated(template_pptx_path, chart_file_pairs, output_pptx_path, slide_idx=1):
    prs = Presentation(template_pptx_path)
    template_slide = prs.slides[slide_idx]
    positions = get_placeholder_positions(template_pptx_path, slide_idx)

    blank_layout = template_slide.slide_layout  # use the same layout

    for peer_group_img, peer_avg_img in chart_file_pairs:
        slide = prs.slides.add_slide(blank_layout)
        for idx, img_path in enumerate([peer_group_img, peer_avg_img]):
            if idx < len(positions):
                left, top, width, height = positions[idx]
                slide.shapes.add_picture(img_path, left, top, width=width, height=height)

    prs.save(output_pptx_path)
    print(f"Saved automated deck: {output_pptx_path}")

# Usage example (run this after upload/check/graph generation):
add_graphs_automated(pptx_filename, chart_file_pairs, "Deck_with_Graphs.pptx", slide_idx=1)


FileNotFoundError: [Errno 2] No such file or directory: 'charts/MCD_3.500pct_7_2027_peer_group.png'

In [ ]:
import os
print("Current directory files:", os.listdir())
if "Event_Template.pptx" not in os.listdir():
    print("Uploading template PPTX...")
    from google.colab import files
    uploaded = files.upload()
    print("Uploaded files:", uploaded)
    # Adjust filename if user uploads a different name
    pptx_filename = next(iter(uploaded))
else:
    pptx_filename = "Event_Template.pptx"

# Now build the deck using the actual filename
add_graphs_automated(pptx_filename, chart_file_pairs, "Deck_with_Graphs.pptx", slide_idx=1)

print("Final files:", os.listdir())


Current directory files: ['.config', 'PortraitTemplate (1).pptx', 'PortraitTemplate.pptx', 'Raw', 'charts', 'sample_data']
Uploading template PPTX...


Saving Event_Template.pptx to Event_Template.pptx
Uploaded files: {'Event_Template.pptx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00Z\xb9u8\xb4\x01\x00\x00I\t\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

AttributeError: 'CT_Picture' object has no attribute 'clone'

In [ ]:
from google.colab import files
files.download("Deck_with_Graphs.pptx")
